<b><h1 style="color:blue;">COURSERA IBM DATA SCIENCCE - CAPSTONE</h1></b>

This notebook is for the IBM Data Science Capstone Project on Coursera
______________________________________________________________________

In [1]:
import numpy as np
import pandas as pd

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


<b>1. Webscrapping - Torronto Neighborhoods Data</b>

In [3]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

In [4]:
#get url and create bs object
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = requests.get(url).text
nbhoods_bs = BeautifulSoup(data,"html.parser")  

<details><summary>Tips</summary>

```**Tips for  Webscraping Updated Table in Week3 Peer Graded Assignment

**After retreiving the URL and creating a Beautiful soup object** 

**Firstly create a list**  

**Later after finding the table and table data  create a dictionary called cell having 3 keys PostalCode, Borough and Neighborhood.**

**As postal code contains upto 3 characters extract that using tablerow.p.text**

**Next use split ,strip and replace functions for getting Borough and Neighborhood information.**.

**Append to the list**  

**Create a dataframe with list**
```

</details>



In [5]:
table_contents = []
toronto_table = nbhoods_bs.find('table')
for row in toronto_table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
nbhoods_df = pd.DataFrame(table_contents)
nbhoods_df['Borough'] = nbhoods_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

nbhoods_df.tail()


,Postal Code,Borough,Neighborhood
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [6]:
nbhoods_df.shape

(103, 3)

<b>2. Use Geocoder to get coordinates for each borough</b>

In [7]:
#!pip install geocoder
import geocoder


<details><summary>Doesnt work</summary>

```
</details># initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
postal_code = nbhoods_df['PostalCode']

while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format('postal_code'))
    lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

nbhoods_df['Latitude'] = latitude
nbhoods_df['Longitude'] = longitude

nbhoods_df.head(10)
```

In [8]:
#import coord file
coord = pd.read_csv('Geospatial_Coordinates.csv')
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
nbhoods_df = nbhoods_df.merge(coord, on=['Postal Code'])
nbhoods_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


<b>3. Explore and cluster Toronto neighborhoods</b>

In [10]:
import json
from geopy.geocoders import Nominatim

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

In [11]:
#Get toronto coords
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Toronto map using latitude and longitude values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lon, borough, neighborhood in zip(nbhoods_df['Latitude'], nbhoods_df['Longitude'], nbhoods_df['Borough'], nbhoods_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

In [12]:
#Defind Foursquare keys
CLIENT_ID = 'FQ5LPE3QAW1NWMBYVEATZWOUA0YLQRODWMIG1M1L00FG2HWL' 
CLIENT_SECRET = '331B2HD2W2OPGOPQIYDE5QG40UDWA0CI3BFOP23JAKSWQY0U'
ACCESS_TOKEN = 'USVMVYZ50B2RBHSBUUVQCES1BQL5ZKOUA0JGIHKS4YLLNBN2'
VERSION = '20210505'
LIMIT = 100

In [13]:
#Get venues in th eneighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
#apply function to df
toronto_venues = getNearbyVenues(names=nbhoods_df['Neighborhood'],
                                   latitudes=nbhoods_df['Latitude'],
                                   longitudes=nbhoods_df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [15]:
toronto_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2131,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Royal Canadian Legion #210,43.628855,-79.518903,Social Club
2132,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Koala Tan Tanning Salon & Sunless Spa,43.631370,-79.519006,Tanning Salon
2133,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Value Village,43.631269,-79.518238,Thrift / Vintage Store
2134,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Kingsway Boxing Club,43.627254,-79.526684,Gym
2135,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Burrito Boyz,43.626657,-79.526349,Burrito Place


In [16]:
#check how many entries in each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
...,...,...,...,...,...,...
Willowdale South,34,34,34,34,34,34
Willowdale West,6,6,6,6,6,6
Woburn,3,3,3,3,3,3


In [17]:
#number of unique categotries 
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 275 uniques categories.


In [18]:
#Analyze each neighborhood:
# one hot encoding
toronto_temp = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_temp['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_temp.columns[-1]] + list(toronto_temp.columns[:-1])
toronto_temp = toronto_temp[fixed_columns]

toronto_temp.tail()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
2131,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2132,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2133,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2134,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2135,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
#Group entry by neighborhood
toronto_grouped = toronto_temp.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Willowdale South,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.029412,0.0,0.0,0.0,0.0
95,Willowdale West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
96,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
97,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [20]:
toronto_grouped.shape

(99, 275)

In [21]:
#function to sort venues in popularity order
def most_common_venues(row, num_top_venues):
    row_cat = row.iloc[1:]
    row_cat_sorted = row_cat.sort_values(ascending = False)
    
    return row_cat_sorted.index.values[0:num_top_venues]


In [22]:
#get 10 top venus for each neighborhood
num_top_venues = 10

indicators = ['st','nd','rd']

#create columns
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
#create new dataframe for 10 top venues
toronto_nbh_venues_sorted = pd.DataFrame(columns = columns)
toronto_nbh_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_nbh_venues_sorted.iloc[ind, 1:] = most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    
toronto_nbh_venues_sorted.tail()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Willowdale South,Ramen Restaurant,Coffee Shop,Pizza Place,Shopping Mall,Café,Sushi Restaurant,Movie Theater,Juice Bar,Korean Restaurant,Restaurant
95,Willowdale West,Pharmacy,Grocery Store,Supermarket,Discount Store,Coffee Shop,Pizza Place,Other Great Outdoors,Organic Grocery,Medical Center,Mediterranean Restaurant
96,Woburn,Coffee Shop,Korean BBQ Restaurant,Yoga Studio,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop
97,Woodbine Heights,Skating Rink,Spa,Park,Athletics & Sports,Beer Store,Curling Ice,Bus Stop,Miscellaneous Shop,Molecular Gastronomy Restaurant,Modern European Restaurant
98,York Mills West,Electronics Store,Park,Convenience Store,Yoga Studio,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop


In [37]:
#Cluster neighborhoods

# set number of clusters
kclusters = 8

toronto_clustered = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustered)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
toronto_nbh_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_nbh_venues_sorted['Cluster Labels'] = toronto_nbh_venues_sorted['Cluster Labels'].fillna(0.0).astype(int)

toronto_merged = nbhoods_df

# merge toronto_grouped with nbhoods to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_nbh_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].fillna(0.0).astype(int)

toronto_merged.tail()

ValueError: cannot insert Cluster Labels, already exists

In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

IndexError: list index out of range

In [39]:
#Clustering neghborhoods
#Cluster 1
print('Cluster 1:')
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 1:


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,Scarborough,0,Coffee Shop,Korean BBQ Restaurant,Yoga Studio,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop
45,North York,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,North York,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,Scarborough,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
#Cluster 2
print('Cluster 2:')
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 2:


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,York,1,Park,Women's Store,Pool,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
35,East York/East Toronto,1,Park,Convenience Store,Yoga Studio,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
40,North York,1,Airport,Park,Yoga Studio,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
64,York,1,Park,Jewelry Store,Yoga Studio,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Mexican Restaurant
66,North York,1,Electronics Store,Park,Convenience Store,Yoga Studio,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
85,Scarborough,1,Intersection,Playground,Park,Miscellaneous Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
91,Downtown Toronto,1,Park,Playground,Trail,Yoga Studio,Mexican Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop


In [41]:
#Cluster 3
print('Cluster 3:')
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 3:


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,2,Pizza Place,Financial or Legal Service,Intersection,Hockey Arena,Portuguese Restaurant,Coffee Shop,Yoga Studio,Mobile Phone Shop,Moroccan Restaurant,Monument / Landmark
8,East York,2,Pizza Place,Flea Market,Athletics & Sports,Pet Store,Breakfast Spot,Bank,Gastropub,Intersection,Pharmacy,Gym / Fitness Center
17,Etobicoke,2,Pizza Place,Beer Store,Shopping Plaza,Liquor Store,Coffee Shop,Pet Store,Pharmacy,Café,Metro Station,Men's Store
63,York,2,Pizza Place,Convenience Store,Bus Line,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant
70,Etobicoke,2,Playground,Coffee Shop,Middle Eastern Restaurant,Sandwich Place,Discount Store,Intersection,Chinese Restaurant,Pizza Place,Optical Shop,Opera House
72,North York,2,Pharmacy,Grocery Store,Supermarket,Discount Store,Coffee Shop,Pizza Place,Other Great Outdoors,Organic Grocery,Medical Center,Mediterranean Restaurant
82,Scarborough,2,Pizza Place,Pharmacy,Fast Food Restaurant,Convenience Store,Noodle House,Fried Chicken Joint,Bank,Chinese Restaurant,Italian Restaurant,Thai Restaurant
89,Etobicoke,2,Grocery Store,Pizza Place,Sandwich Place,Pharmacy,Beer Store,Liquor Store,Fried Chicken Joint,Fast Food Restaurant,Optical Shop,Opera House
90,Scarborough,2,Fast Food Restaurant,Pharmacy,Intersection,Pizza Place,Electronics Store,Supermarket,Bank,Sandwich Place,Chinese Restaurant,Discount Store
93,Etobicoke,2,Pizza Place,Sandwich Place,Gym,Playground,Pool,Coffee Shop,Pub,Yoga Studio,Mobile Phone Shop,Moroccan Restaurant


In [42]:
#Cluster 4
print('Cluster 4:')
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 4:


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,3,Fast Food Restaurant,Yoga Studio,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Mexican Restaurant


In [43]:
#Cluster 5
print('Cluster 5:')
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 5:


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4,Fast Food Restaurant,Park,Food & Drink Shop,Yoga Studio,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
49,North York,4,Basketball Court,Construction & Landscaping,Bakery,Park,Yoga Studio,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
61,Central Toronto,4,Construction & Landscaping,Swim School,Park,Bus Line,Yoga Studio,Miscellaneous Shop,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
77,Etobicoke,4,Pizza Place,Park,Bus Line,Sandwich Place,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
83,Central Toronto,4,Restaurant,Trail,Park,Summer Camp,Yoga Studio,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant


In [44]:
#Cluster 6
print('Cluster 6:')
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 6:


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,5,Coffee Shop,Park,Bakery,Breakfast Spot,Theater,Pub,Café,Chocolate Shop,Beer Store,French Restaurant
3,North York,5,Clothing Store,Furniture / Home Store,Women's Store,Coffee Shop,Miscellaneous Shop,Boutique,Vietnamese Restaurant,Accessories Store,Park,Pet Store
4,Queen's Park,5,Coffee Shop,Sushi Restaurant,Yoga Studio,Japanese Restaurant,Burrito Place,Burger Joint,Smoothie Shop,Café,Spa,Fried Chicken Joint
7,North York,5,Caribbean Restaurant,Café,Gym,Dessert Shop,Japanese Restaurant,Yoga Studio,Molecular Gastronomy Restaurant,Motel,Moroccan Restaurant,Monument / Landmark
9,Downtown Toronto,5,Coffee Shop,Clothing Store,Cosmetics Shop,Bubble Tea Shop,Italian Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Café,Hotel,Electronics Store
...,...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,5,Coffee Shop,Café,Hotel,Japanese Restaurant,Gym,Restaurant,Salad Place,Deli / Bodega,Steakhouse,American Restaurant
98,Etobicoke,5,River,Pool,Smoke Shop,Yoga Studio,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
99,Downtown Toronto,5,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Mediterranean Restaurant,Men's Store,Fast Food Restaurant,Dance Studio
100,East Toronto Business,5,Light Rail Station,Brewery,Restaurant,Recording Studio,Garden Center,Skate Park,Burrito Place,Farmers Market,Spa,Fast Food Restaurant


In [45]:
#Cluster 7
print('Cluster 7:')
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 7:


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Scarborough,6,Playground,Yoga Studio,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Mexican Restaurant


In [46]:
#Cluster 8
print('Cluster 8:')
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 8:


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Scarborough,7,History Museum,Bar,Yoga Studio,Mobile Phone Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Miscellaneous Shop


In [47]:
#Cluster 9
print('Cluster 9:')
toronto_merged.loc[toronto_merged['Cluster Labels'] == 8, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 9:


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,North York,8,Food Truck,Baseball Field,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Yoga Studio,Motel
57,North York,8,Baseball Field,Yoga Studio,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant,Movie Theater
101,Etobicoke,8,Pool,Baseball Field,Yoga Studio,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Middle Eastern Restaurant


In [48]:
#Cluster 10
print('Cluster 10:')
toronto_merged.loc[toronto_merged['Cluster Labels'] == 9, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 10:


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,Scarborough,9,American Restaurant,Motel,Museum,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Yoga Studio


<h1>END</h1>